In [ ]:
# installing torchtext packagge 0.8
!pip install torchtext==0.8

     |████████████████████████████████| 6.9 MB 6.3 MB/s 
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.12.0
    Uninstalling torchtext-0.12.0:
      Successfully uninstalled torchtext-0.12.0


In [ ]:
# import package
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.datasets import TranslationDataset
from torchtext.data import Field,TabularDataset,Iterator,BucketIterator

from spacy.lang.en import English
from spacy.lang.de import German
from spacy.lang.fr import French
import spacy
import numpy as np

import random
import math
import time

import unicodedata
import string
import re
import random

In [ ]:
## Mount Google Drive Data (If using Google Colaboratory)
try:
    from google.colab import drive
    drive.mount('/content/gdrive')
except:
    print("Mounting Failed.")

Mounted at /content/gdrive


Process zip file

In [ ]:
# unzip data
import zipfile
import os
from tqdm import tqdm, trange

! tar xvf /content/gdrive/MyDrive/Colab\ Notebooks/ml/hansard.36.r2001-1a.house.debates.training.tar
! tar xvf /content/gdrive/MyDrive/Colab\ Notebooks/ml/hansard.36.r2001-1a.house.debates.testing.tar
% cd /content/hansard.36/Release-2001.1a/sentence-pairs/house/debates/development/testing/1
! gunzip *.gz
% cd /content/hansard.36/Release-2001.1a/sentence-pairs/house/debates/development/testing/2
! gunzip *.gz
% cd /content/hansard.36/Release-2001.1a/sentence-pairs/house/debates/development/training
! gunzip *.gz


hansard.36/Release-2001.1a/sentence-pairs/house/debates/development/training/
hansard.36/Release-2001.1a/sentence-pairs/house/debates/development/training/hansard.36.1.house.debates.001.f.gz
hansard.36/Release-2001.1a/sentence-pairs/house/debates/development/training/hansard.36.1.house.debates.002.e.gz
hansard.36/Release-2001.1a/sentence-pairs/house/debates/development/training/hansard.36.1.house.debates.002.f.gz
hansard.36/Release-2001.1a/sentence-pairs/house/debates/development/training/hansard.36.1.house.debates.003.e.gz
hansard.36/Release-2001.1a/sentence-pairs/house/debates/development/training/hansard.36.1.house.debates.003.f.gz
hansard.36/Release-2001.1a/sentence-pairs/house/debates/development/training/hansard.36.1.house.debates.004.e.gz
hansard.36/Release-2001.1a/sentence-pairs/house/debates/development/training/hansard.36.1.house.debates.004.f.gz
hansard.36/Release-2001.1a/sentence-pairs/house/debates/development/training/hansard.36.1.house.debates.005.e.gz
hansard.36/Release

Load data

In [ ]:
# load data file
dir = '/content/hansard.36/Release-2001.1a/sentence-pairs/house/debates/development/{}ing/'
def load_data(mode):
    if mode == 'train':
        file_dir = dir.format('train')
    elif mode == 'dev':
        file_dir = dir.format('test') + '1/'
    elif mode == 'test':
        file_dir = dir.format('test') + '2/'
    else:
        raise Exception("select mode: train / dev / test")
        
    filenames = []
    fr_files = []
    en_files = []
    # for filename in os.listdir(file_dir):
    for filename in os.listdir(file_dir):
        if filename.split('.')[-1] == 'e':
          myfile = '.'.join(filename.split('.')[:-1])
          fr_files.append(myfile+'.f')
          en_files.append(myfile+'.e')

    
    en_data = load_from_sorted_files(en_files, file_dir)
    fr_data = load_from_sorted_files(fr_files, file_dir)
    print('en sentences:', len(en_data))
    print('fr sentences:', len(fr_data))
    return [en_data, fr_data]

def load_from_sorted_files(files, file_dir):
    dataset = []
    for file in tqdm(files):
        try:
          with open(file_dir+file, 'r', encoding='latin-1') as f:
              lines = f.readlines()
              for i in range(len(lines)):
                  line = lines[i]
                  line = line.strip()

                  dataset.append(line)
        except:
          continue
    return dataset
en_data_tr, fr_data_tr = load_data('train')
en_data_dev, fr_data_dev = load_data('dev')
en_data_te, fr_data_te = load_data('test')

100%|██████████| 313/313 [00:00<00:00, 392.21it/s]


en sentences: 947969
fr sentences: 947969


100%|██████████| 22/22 [00:00<00:00, 558.33it/s]


en sentences: 62213
fr sentences: 62213


100%|██████████| 15/15 [00:00<00:00, 466.21it/s]

en sentences: 60077
fr sentences: 60077


In [ ]:
spacy_en = English()
spacy_fr = French()
def tokenize_fr(text):
    return [tok.text for tok in spacy_fr.tokenizer(text)][::-1]

def tokenize_en(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]

Define field

In [ ]:
SRC_field = Field(tokenize = tokenize_fr, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

TRG_field = Field(tokenize = tokenize_en, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

/usr/local/lib/python3.7/dist-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


process data

In [ ]:
import pandas as pd

d= []
count = 0
for i in range(len(en_data_tr)):
  # if count>100:
  #   break

  if len(en_data_tr[i].split(' '))>=60:
    continue
  # count += 1
  d.append([fr_data_tr[i],en_data_tr[i]])

train = pd.DataFrame(data=d,
             columns = ['fr','en'])
train.to_csv("./train.csv", index=False)

d= []
for i in range(len(en_data_dev)):
  # if i>100:
  #   break
  if len(en_data_dev[i].split(' '))>=60:
    continue
  d.append([fr_data_dev[i],en_data_dev[i]])

test = pd.DataFrame(data=d,
             columns = ['fr','en'])
test.to_csv("./val.csv", index=False)

d= []
for i in range(len(en_data_te)):
  # if i>100:
  #   break
  if len(en_data_te[i].split(' '))>=60:
    continue
  d.append([fr_data_te[i],en_data_te[i]])

test = pd.DataFrame(data=d,
             columns = ['fr','en'])
test.to_csv("./test.csv", index=False)
fields = [('fr', SRC_field), ('en', TRG_field)]

train_data, valid_data, test_data= TabularDataset.splits(path='./', train='train.csv', validation='val.csv', test = 'test.csv', format='csv', fields=fields)

/usr/local/lib/python3.7/dist-packages/torchtext/data/example.py:68: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)
/usr/local/lib/python3.7/dist-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)


build vocabary

In [ ]:
SRC_field.build_vocab(train_data, min_freq = 2)
TRG_field.build_vocab(train_data, min_freq = 2)

Define encoder, decoder and seq2seq model

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.hid_dim = hid_dim
        self.n_layers = n_layers        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        self.dropout = nn.Dropout(dropout)
        self.embedding = nn.Embedding(input_dim, emb_dim)

    def forward(self, src):
        embedded = self.dropout(self.embedding(src))
        outputs, (hidden, cell) = self.rnn(embedded)
        return hidden, cell


class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        self.fc_out = nn.Linear(hid_dim, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, cell):
        input = input.unsqueeze(0)
        embedded = self.dropout(self.embedding(input))
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        prediction = self.fc_out(output.squeeze(0))
        return prediction, hidden, cell

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):

        batch_size = trg.shape[1]
        trg_length = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        outputs = torch.zeros(trg_length, batch_size, trg_vocab_size).to(self.device)
        
        hidden, cell = self.encoder(src)
        input = trg[0,:]
        
        for t in range(1, trg_length):

            output, hidden, cell = self.decoder(input, hidden, cell)
            outputs[t] = output

            if random.random() < teacher_forcing_ratio:
              input = trg[t]
            else:
              input = output.argmax(1)
        return outputs

Initialization

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
INPUT_DIM = len(SRC_field.vocab)
OUTPUT_DIM = len(TRG_field.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.1
DEC_DROPOUT = 0.1

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)


model = Seq2Seq(enc, dec, device).to(device)
optimizer = optim.Adam(model.parameters())
TRG_PAD_IDX = TRG_field.vocab.stoi[TRG_field.pad_token]
criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.1)
    (dropout): Dropout(p=0.1, inplace=False)
    (embedding): Embedding(50561, 256)
  )
  (decoder): Decoder(
    (embedding): Embedding(38701, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.1)
    (fc_out): Linear(in_features=512, out_features=38701, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
)

Build dataloader

In [ ]:
BATCH_SIZE = 128
train_iterator = BucketIterator(
    train_data, 
    batch_size = BATCH_SIZE, 
    shuffle=True,
    device = device)
valid_iterator = BucketIterator(
    valid_data, 
    batch_size = BATCH_SIZE, 
    device = device)
test_iterator = BucketIterator(
    test_data, 
    batch_size = BATCH_SIZE, 
    device = device)

/usr/local/lib/python3.7/dist-packages/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


Train and evaluate

In [ ]:
def train(model, iterator, optimizer = optimizer, criterion = criterion, clip = 1):
    
    model.train()
    loss_epochs = 0
    
    for i, batch in enumerate(iterator):
        loss = 0

        src_data = batch.fr
        trg_data = batch.en
        
        optimizer.zero_grad()
        
        output = model(src_data, trg_data)
        output = output[1:].view(-1, output.shape[-1])

        trg_data = trg_data[1:].view(-1)

        
        loss = criterion(output, trg_data)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        loss_epochs += loss.item()

    return loss_epochs / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion = criterion):
    
    model.eval()
    
    loss_epoch = 0
    
    with torch.no_grad():
        for i, batch in enumerate(iterator):
            src_data = batch.fr
            trg_data = batch.en

            output = model(src_data, trg_data, 0)
            output = output[1:].view(-1, output.shape[-1])

            trg_data = trg_data[1:].view(-1)


            loss = criterion(output, trg_data)
            loss_epoch += loss.item()
        
    return loss_epoch / len(iterator)

In [ ]:
def cal_time_cost(start, end):
    cost = end - start
    minutes = int(cost / 60)
    seconds = int(cost - (minutes * 60))
    return minutes, seconds

Train model

In [ ]:
N_EPOCHS = 8

best_valid_loss = float('inf')
total_train_loss = []
total_valid_loss = []


for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator)
    total_train_loss.append(train_loss)
    valid_loss = evaluate(model, valid_iterator)
    total_valid_loss.append(valid_loss)

    for p in optimizer.param_groups:
      p['lr'] *=0.8
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = cal_time_cost(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), '/content/gdrive/MyDrive/Colab Notebooks/ml/lstm_RNN_Based_Model.pt')
    
    print(f'Epoch: {epoch+1:02}, Time: {epoch_mins}m {epoch_secs}s')
    print(f',Train Loss is: {train_loss:.3f}')
    print(f',Validation Loss: {valid_loss:.3f}')

/usr/local/lib/python3.7/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


Epoch: 01, Time: 126m 31s
,Train Loss is: 4.615
,Validation Loss: 4.879
Epoch: 02, Time: 126m 39s
,Train Loss is: 3.799
,Validation Loss: 4.594
Epoch: 03, Time: 126m 19s
,Train Loss is: 3.504
,Validation Loss: 4.488
Epoch: 04, Time: 126m 22s
,Train Loss is: 3.321
,Validation Loss: 4.422
Epoch: 05, Time: 126m 27s
,Train Loss is: 3.197
,Validation Loss: 4.408
Epoch: 06, Time: 126m 20s
,Train Loss is: 3.109
,Validation Loss: 4.343
Epoch: 07, Time: 126m 18s
,Train Loss is: 3.041
,Validation Loss: 4.329
Epoch: 08, Time: 126m 20s
,Train Loss is: 2.986
,Validation Loss: 4.303


Test Model

In [ ]:
model.load_state_dict(torch.load('/content/gdrive/MyDrive/Colab Notebooks/ml/lstm_RNN_Based_Model.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'Test Loss is: {test_loss:.3f} ')

/usr/local/lib/python3.7/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


Test Loss is: 3.705 


In [ ]:
def translate_sentence(sentence, src_field, trg_field, model, device, max_len = 50):
    model.eval()      
    if isinstance(sentence, str):
        tokens = [token.text.lower() for token in spacy_fr(sentence)]
    else:
        tokens = [token.lower() for token in sentence]

    tokens = [src_field.init_token] + tokens + [src_field.eos_token]
        
    src_indexes = [src_field.vocab.stoi[token] for token in tokens]
    
    src_tensor = torch.LongTensor(src_indexes).unsqueeze(1).to(device)

    src_len = torch.LongTensor([len(src_indexes)])
    
    with torch.no_grad():
        hidden, cell = model.encoder(src_tensor)
        
    trg_indexes = [trg_field.vocab.stoi[trg_field.init_token]]

    for i in range(max_len):

        trg_tensor = torch.LongTensor([trg_indexes[-1]]).to(device)
                
        with torch.no_grad():
            output, hidden, cell = model.decoder(trg_tensor, hidden, cell)
            
        pred_token = output.argmax(1).item()
        
        trg_indexes.append(pred_token)

        if pred_token == trg_field.vocab.stoi[trg_field.eos_token]:
            break
    
    trg_tokens = [trg_field.vocab.itos[i] for i in trg_indexes]
    
    return trg_tokens[1:]

Calculate BLEU Score

In [ ]:
from torchtext.data.metrics import bleu_score

def calculate_bleu(data, src_field, trg_field, model, device, max_len = 50):
    
    trgs = []
    pred_trgs = []
    
    for datum in data:
        
        src = vars(datum)['fr']
        trg = vars(datum)['en']
        
        pred_trg= translate_sentence(src, src_field, trg_field, model, device, max_len)
        
        #cut off <eos> token
        pred_trg = pred_trg[:-1]
        
        pred_trgs.append(pred_trg)
        trgs.append([trg])
        
    return bleu_score(pred_trgs, trgs)
bleu_score = calculate_bleu(test_data, SRC_field, TRG_field, model, device)
print(f'BLEU score = {bleu_score*100:.2f}')

BLEU score = 18.64


BLEU score is 18.64